In [2]:
import numpy as np
import nltk
from nltk.stem.lancaster import LancasterStemmer
import json
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import csv
import pandas as pd
import re

In [4]:
def sigmoid(x):
    return (1/(1+np.exp(-x)))

print(sigmoid(np.array([1,2,22,4])))

[0.73105858 0.88079708 1.         0.98201379]


In [8]:
def softmax(x):
    exp_x = np.exp(x)
    return exp_x/exp_x.sum(axis=0)

print(softmax(np.array([2,3,4,5])))

[0.0320586  0.08714432 0.23688282 0.64391426]


In [3]:
def json_to_data(jsonfile, csvfile):
    file = open(jsonfile)
    data = json.load(file)
    
    f = csv.writer(open(csvfile, 'w'))
    f.writerow(["emotion","utterance"])
    
    for item in data:
        for line in item:
            f.writerow([line['emotion'],line['utterance']])
       

In [6]:
json_to_data('DataSets/Friends/friends_train.json', 'DataSets/Friends/textData.csv')

In [7]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
regex = re.compile('[^a-zA-Z\' ]')

def cleaner(string):
    #string = regex.sub('',string)
    string = string.translate({ord(i): None for i in '!.,&%#@?;\\'})
    string = string.replace('-', ' ')
    try:
        w = string.lower().split()
    except:
        return "NAW"
    w = [p.lower() for p in w]
    
    #w = [lemmatizer.lemmatize(word) for word in w]
    string = " ".join(w)
    return string


In [8]:
def prepare_data(csvfile):
    file = pd.read_csv(csvfile)
    cols = ['emotion', 'utterance']
    file = file[cols]
    file['emotion_id'] = file['emotion'].factorize()[0]
    file['utterance'] = [cleaner(sent) for sent in file['utterance']]
    file = file[pd.notnull(file['utterance'])]
    return file['utterance'], file['emotion_id']

In [9]:
X,Y = prepare_data('DataSets/Friends/textData.csv')
print(X, Y)


0        also i was the point person on my companys tr...
1                          you mustve had your hands full
2                                    that i did that i did
3             so lets talk a little bit about your duties
4                                      my duties all right
5        now youll be heading a whole division so you...
6                                                    i see
7        but therell be perhaps 30 people under you so...
8                                             good to know
9                                    we can go into detail
10                                   no dont i beg of you
11       all right then well have a definite answer fo...
12                                                  really
13                  absolutely you can relax you did great
14       but then who the waitress i went out with last...
15                                      you know forget it
16           no no no no no who who were you talking abo

In [11]:
print(zip(X, Y))
f = csv.writer(open('DataSets/Friends/ProcessedData.csv', 'w'))
f.writerow(["emotion","utterance"])
    
for item in zip(Y,X):
    f.writerow(item)

In [33]:
def read_glove_vecs(embed):
    with open(embed, 'r') as f:
        words = set()
        word_to_vec_map = {}
        j=0
        for line in f:
            j+=1
            line = line.strip().split()
            word = line[0]
            words.add(word)
            word_to_vec_map[word] = np.array(line[1:], dtype=np.float64)
            
        i=1
        words_to_index = {}
        index_to_words = {}
        index_to_vec = np.zeros((j,50))
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            index_to_vec[i-1,:] = word_to_vec_map[w]
            i+=1
        return j, words_to_index, index_to_words, word_to_vec_map, index_to_vec
            

In [34]:
num_words, words_to_index, index_to_words, word_to_vec_map, index_to_vec = read_glove_vecs('DataSets/glove.6B.50d.txt')

In [265]:
maxi = 0
for p in X:
    w = p.lower().split()
    j=0
    for i in w:
        try:
            a = word_to_vec_map[i]
        except KeyError:
            continue
        j+=1
    maxi = max(j,maxi)

print(maxi)

66


In [15]:
def get_X_mat(X):
    Xmat = np.zeros((X.shape[0],50))
    for i in range(X.shape[0]):
        try:
            words = X[i].lower().split()
        except:
            continue
        j=0
        for word in words:
            try:
                Xmat[i,j] = words_to_index[word]
            except KeyError:
                continue
            j+=1
            if j>49:
                break
    return Xmat  
    

In [17]:
Xmat = get_X_mat(X)
print(Xmat.shape)

(10561, 50)


In [25]:
from keras.models import Model
from keras.layers import Dense, RNN, Input, Dropout, LSTM, Activation, Embedding
from keras.utils import to_categorical
from keras.optimizers import Adam

In [19]:
Ymat = to_categorical(Y)
print(Ymat.shape)


(10561, 8)


In [35]:
def RNN_Model_Keras(input_shape):
    inputs = Input(shape=(input_shape), dtype=np.float32)
    X = Embedding(400000,50,weights=[index_to_vec], input_length=50)(inputs)
    X = LSTM(128)(X)
    X = Dense(8, activation='softmax')(X)
    model = Model(inputs=inputs, outputs=X)
    return model

In [36]:
model = RNN_Model_Keras((50,))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 50, 50)            20000000  
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 1032      
Total params: 20,092,680
Trainable params: 20,092,680
Non-trainable params: 0
_________________________________________________________________


In [328]:
ad = Adam(lr=0.005)
model.compile(loss='categorical_crossentropy', optimizer=ad, metrics=['accuracy'])

In [ ]:
model.fit(Xmat, Ymat, epochs = 25, batch_size = 5, shuffle=True)

Epoch 1/25
  830/10561 [=>............................] - ETA: 2:45 - loss: nan - acc: 0.4819

In [299]:
dev_file = pd.read_csv('Project/DataSets/CompProcessedDataDev.csv')
cols = ['emotion', 'utterance']
dev_file = dev_file[pd.notnull(dev_file['utterance'])]
X_test = dev_file['utterance']
Y_test = dev_file['emotion']

In [300]:
print(type(X[0]))
Xmat_test = get_X_mat(X_test)
print(Xmat_test.shape)
Ymat_test = to_categorical(Y_test)
print(Ymat_test.shape)

<class 'str'>
(972, 50, 50)
(972, 8)


In [306]:
loss, acc = model.evaluate(Xmat_test, Ymat_test)
print(loss)
print(acc)

972/972 [==============================] - 1s 1ms/step
3.3514347164719194
0.33436214004033876
